In [1]:
import pyspark
from pyspark import SQLContext

In [2]:
from IPython.core.display import HTML
display(HTML(""))

In [4]:
sc = SparkSession.builder.appName('Testing').getOrCreate()
sqlContext = SQLContext(sc)

In [5]:
emp = [(1,"Smith",-1,"2018","10","M",3000), \
    (2,"Rose",1,"2010","20","M",4000), \
    (3,"Williams",1,"2010","10","M",1000), \
    (4,"Jones",2,"2005","10","F",2000), \
    (5,"Brown",2,"2010","40","",-1), \
      (6,"Brown",2,"2010","50","",-1) \
  ]
empColumns = ["emp_id","name","superior_emp_id","year_joined", \
       "emp_dept_id","gender","salary"]

empDF = spark.createDataFrame(data=emp, schema = empColumns)
empDF.printSchema()
empDF.show(truncate=False)

dept = [("Finance",10), \
    ("Marketing",20), \
    ("Sales",30), \
    ("IT",40) \
  ]
deptColumns = ["dept_name","dept_id"]
deptDF = spark.createDataFrame(data=dept, schema = deptColumns)
deptDF.printSchema()
deptDF.show(truncate=False)


root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
|6     |Brown   |2              |2010       |50         |      |-1    |
+------+--------+---------------+-----------+-----------+------+-----

In [6]:
def inputValues(values):
    replace_new_line = values.replace('\n', ',')     # replace new line with comma
    input_value_to_list = replace_new_line.split(',')       # input values to list
    empty_space = [x for x in input_value_to_list if x]      # remove empty space in list
    result = [eval(i) for i in empty_space]        # convert list from string to integer
    return result

In [8]:
values = '10'

insertData = inputValues(values)

In [10]:
filtering = empDF.filter(empDF.emp_dept_id.isin(insertData))
filtering = filtering.select(filtering['emp_id'], filtering['name'], filtering['emp_dept_id'])

In [12]:
empDF_sql = filtering.createOrReplaceTempView('filtering')
deptDF_sql = deptDF.createOrReplaceTempView('deptDF')

In [15]:
joinDF = spark.sql('''
select * from filtering a 
LEFT JOIN deptDF b 
ON a.emp_dept_id == b.dept_id
''')

In [16]:
joinDF.show()

+------+--------+-----------+---------+-------+
|emp_id|    name|emp_dept_id|dept_name|dept_id|
+------+--------+-----------+---------+-------+
|     1|   Smith|         10|  Finance|     10|
|     3|Williams|         10|  Finance|     10|
|     4|   Jones|         10|  Finance|     10|
+------+--------+-----------+---------+-------+



In [17]:
joinDF.toPandas()

,emp_id,name,emp_dept_id,dept_name,dept_id
0,1,Smith,10,Finance,10
1,3,Williams,10,Finance,10
2,4,Jones,10,Finance,10
